In [5]:
import re
import os
import random

class TrackManager:
    """一个带有安全车距的智能轨道管理器，确保弹幕流动平稳、不追尾。"""
    def __init__(self, num_tracks, video_width, speed, font_size, safety_buffer):
        self.track_free_time = [0.0] * num_tracks
        self.num_tracks = num_tracks
        self.video_width = video_width
        self.speed = speed
        self.font_size = font_size
        self.safety_buffer = safety_buffer # 安全车距（秒）

    def _calculate_text_width(self, text):
        width = 0
        for char in text:
            if '\u4e00' <= char <= '\u9fff' or '\u3040' <= char <= '\u30ff' or '\uff00' <= char <= '\uffef':
                width += self.font_size
            else:
                width += self.font_size * 0.6
        return width

    def place_danmaku(self, original_start_time, text):
        text_width = self._calculate_text_width(text)
        
        # <<< 终极修复：引入安全车距，重新定义轨道占用时间 >>>
        # 轨道被占用的时间 = (弹幕尾部离开出生点的时间) + (安全车距)
        time_to_clear_spawn_point = (text_width / self.speed) + self.safety_buffer
        
        available_tracks = [i for i, t in enumerate(self.track_free_time) if t <= original_start_time]
        if available_tracks:
            best_track = random.choice(available_tracks)
            start_time = original_start_time
        else:
            earliest_free_time = min(self.track_free_time)
            best_track = self.track_free_time.index(earliest_free_time)
            start_time = earliest_free_time
            
        self.track_free_time[best_track] = start_time + time_to_clear_spawn_point
        
        total_distance = self.video_width + text_width
        duration = total_distance / self.speed
        end_time = start_time + duration
        
        y_pos = (best_track + 1) * (self.font_size + 8)
        
        return start_time, end_time, y_pos, text_width


def lrc_to_ass_final(lrc_file, ass_file, video_width, video_height, speed, font_size, num_tracks, safety_buffer):
    header = f"""[Script Info]
Title: Converted from LRC
ScriptType: v4.00+
PlayResX: {video_width}
PlayResY: {video_height}
WrapStyle: 2

[V4+ Styles]
Format: Name, Fontname, Fontsize, PrimaryColour, SecondaryColour, OutlineColour, BackColour, Bold, Italic, Underline, StrikeOut, ScaleX, ScaleY, Spacing, Angle, BorderStyle, Outline, Shadow, Alignment, MarginL, MarginR, MarginV, Encoding
Style: Danmaku,Segoe UI Emoji, KaiTi,{font_size},&H00FFFFFF,&H00FFFFFF,&H00000000,&H00000000,0,0,0,0,100,100,0,0,1,1.5,0,2,10,10,10,1

[Events]
Format: Layer, Start, End, Style, Name, MarginL, MarginR, MarginV, Effect, Text
"""
    events = []
    lrc_pattern = re.compile(r'\[(\d{2}):(\d{2}):(\d{2})\.(\d{3})\](.*)')

    if not os.path.exists(lrc_file):
        print(f"!!! 错误：找不到弹幕文件 '{lrc_file}'")
        return False

    with open(lrc_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        
    track_manager = TrackManager(num_tracks, video_width, speed, font_size, safety_buffer)

    for line in lines:
        match = lrc_pattern.search(line)
        if not match: continue
        hh, mm, ss, ms, raw_text = match.groups()
        raw_text = raw_text.strip()
        if not raw_text: continue
        
        parts = re.split(r'\s+', raw_text, 1)
        if len(parts) == 2:
            user_id, danmaku_content = parts
            formatted_text = f"{user_id}: {danmaku_content}"
        else:
            formatted_text = raw_text

        original_start_time = int(hh) * 3600 + int(mm) * 60 + int(ss) + int(ms) / 1000
        
        start_time, end_time, y_pos, text_width = track_manager.place_danmaku(original_start_time, formatted_text)
        
        start_h, start_rem = divmod(start_time, 3600); start_m, start_s = divmod(start_rem, 60)
        start_formatted = f"{int(start_h)}:{int(start_m):02}:{int(start_s):02}.{int((start_s % 1) * 100):02}"
        end_h, end_rem = divmod(end_time, 3600); end_m, end_s = divmod(end_rem, 60)
        end_formatted = f"{int(end_h)}:{int(end_m):02}:{int(end_s):02}.{int((end_s % 1) * 100):02}"
        
        effect = f"{{_\\move({video_width}, {y_pos}, -{text_width}, {y_pos})}}"
        final_text = formatted_text.replace('{', '｛').replace('}', '｝')
        events.append(f"Dialogue: 0,{start_formatted},{end_formatted},Danmaku,,0,0,0,,{effect}{final_text}")

    if not events:
        print("\n!!! 严重警告：转换完成，但没有找到任何有效的弹幕行！")
        return False

    with open(ass_file, 'w', encoding='utf-8-sig') as f:
        f.write(header)
        f.write("\n".join(events))
    
    print(f"✅ 成功: LRC 弹幕已转换为 ASS 字幕!\n   文件保存在: {ass_file}\n")
    return True

# --- 主程序 ---
if __name__ == '__main__':
    video_full_path = r"E:\48tool\48kodai\kodaizhibo\[口袋48录播]_CGT48-谭思慧_🌃_2025-06-29~00.17.05_1145874733059608576__2025-07-01~00.17.38.ts"
    lrc_full_path = r"E:\48tool\48kodai\kodaizhibo\[口袋48弹幕]_CGT48-谭思慧_🌃_2025-06-29~00.17.05_1145874733059608576__2025-07-01~00.04.59.lrc"
    
    # --- 在这里精细化配置你的参数 ---
    danmaku_speed = 4
    num_tracks = 15
    font_size = 25

    # <<< 全新参数：安全车距 (秒) >>>
    # 这个值确保了弹幕之间的最小时间间隔，防止追尾。
    # 推荐值: 1.0 到 3.0 之间。
    # 如果你觉得弹幕间距太大，可以调小它；如果还觉得拥挤，可以调大它。
    safety_buffer = 1.5

    output_directory = os.path.dirname(lrc_full_path)
    base_filename = os.path.splitext(os.path.basename(lrc_full_path))[0]
    
    ass_full_path = os.path.join(output_directory, base_filename + ".ass")
    temp_mp4_path = os.path.join(output_directory, base_filename + "_video_only.mp4")
    final_mp4_path = os.path.join(output_directory, base_filename + "_带弹幕.mp4")

    print(f"--- 速度:{danmaku_speed}px/s | 轨道:{num_tracks} | 字体:{font_size}px | 安全距离:{safety_buffer}s ---")
    print("--- 第一步：正在转换LRC为ASS字幕... ---")
    
    if lrc_to_ass_final(lrc_full_path, ass_file=ass_full_path, video_width=1920, video_height=1080, speed=danmaku_speed, font_size=font_size, num_tracks=num_tracks, safety_buffer=safety_buffer):
        print("--- 第二步：请打开命令行，按顺序复制并执行以下FFmpeg命令 ---")
        
        print("\n【命令 1: (推荐) 将TS无损转换为MP4】")
        command1 = f'ffmpeg -i "{video_full_path}" -c copy -y "{temp_mp4_path}"'
        print("复制此行 ->", command1)
        
        print("\n【命令 2: (最终) 合并视频和字幕为软字幕MP4】")
        command2 = f'ffmpeg -i "{temp_mp4_path}" -i "{ass_full_path}" -c copy -c:s mov_text -y "{final_mp4_path}"'
        print("复制此行 ->", command2)
        
        print(f"\n🎉 全部完成后，你的最终带弹幕视频在: {final_mp4_path}")

--- 速度:4px/s | 轨道:15 | 字体:25px | 安全距离:1.5s ---
--- 第一步：正在转换LRC为ASS字幕... ---
✅ 成功: LRC 弹幕已转换为 ASS 字幕!
   文件保存在: E:\48tool\48kodai\kodaizhibo\[口袋48弹幕]_CGT48-谭思慧_🌃_2025-06-29~00.17.05_1145874733059608576__2025-07-01~00.04.59.ass

--- 第二步：请打开命令行，按顺序复制并执行以下FFmpeg命令 ---

【命令 1: (推荐) 将TS无损转换为MP4】
复制此行 -> ffmpeg -i "E:\48tool\48kodai\kodaizhibo\[口袋48录播]_CGT48-谭思慧_🌃_2025-06-29~00.17.05_1145874733059608576__2025-07-01~00.17.38.ts" -c copy -y "E:\48tool\48kodai\kodaizhibo\[口袋48弹幕]_CGT48-谭思慧_🌃_2025-06-29~00.17.05_1145874733059608576__2025-07-01~00.04.59_video_only.mp4"

【命令 2: (最终) 合并视频和字幕为软字幕MP4】
复制此行 -> ffmpeg -i "E:\48tool\48kodai\kodaizhibo\[口袋48弹幕]_CGT48-谭思慧_🌃_2025-06-29~00.17.05_1145874733059608576__2025-07-01~00.04.59_video_only.mp4" -i "E:\48tool\48kodai\kodaizhibo\[口袋48弹幕]_CGT48-谭思慧_🌃_2025-06-29~00.17.05_1145874733059608576__2025-07-01~00.04.59.ass" -c copy -c:s mov_text -y "E:\48tool\48kodai\kodaizhibo\[口袋48弹幕]_CGT48-谭思慧_🌃_2025-06-29~00.17.05_1145874733059608576__2025-07-01~00.04.59_带弹幕.